# GPU를 사용해 학습 가속하기

이번 튜토리얼에서, 다음을 배우게 됩니다.

* 그래프와 피처 데이터를 GPU로 복사하는 방법
* GNN 모델을 GPU 위에 학습하는 방법


In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

Using backend: pytorch


## 그래프와 피처 데이터를 GPU에 복사

먼저 이전 세션에 사용한 Zachery의 카라테 클럽 그래프와 노드 라벨를 로드합니다.

In [2]:
from tutorial_utils import load_zachery

# ----------- 0. load graph -------------- #
g = load_zachery()
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


이제 그래프와 모든 그래프의 피처 데이터는 CPU에 적재되어 있습니다. `to` API를 사용해 다른 연산장치로 복사해 보세요.

In [3]:
print('Current device:', g.device)
g = g.to('cuda:0')
print('New device:', g.device)

Current device: cpu
New device: cuda:0


Verify that features are also copied to GPU.

In [4]:
print(g.ndata['club'].device)
print(g.ndata['club_onehot'].device)

cuda:0
cuda:0


## GNN 모델을 GPU에 생성하기

이 스텝은 CNN이나 RNN 모델을 GPU에 생성하는 것과 같습니다.  
PyTorch에서, `to` API를 사용해 이를 수행할 수 있습니다.

In [5]:
# ----------- 1. node features -------------- #
node_embed = nn.Embedding(g.number_of_nodes(), 5)  # 각 노드는 5차원의 임베딩을 가지고 있습니다.
# Copy node embeddings to GPU
node_embed = node_embed.to('cuda:0')
inputs = node_embed.weight                         # 노드 피처로써 이 임베딩 가중치를 사용합니다.
nn.init.xavier_uniform_(inputs)

Parameter containing:
tensor([[-0.1941,  0.3631, -0.2146, -0.0416, -0.3040],
        [-0.2014,  0.3393, -0.1062, -0.2229,  0.3331],
        [-0.2843,  0.3027,  0.2389, -0.0853,  0.3283],
        [ 0.0482,  0.1986, -0.2904,  0.0818, -0.1860],
        [-0.0844, -0.3876,  0.1654, -0.2600,  0.3482],
        [ 0.1354, -0.1090,  0.0389,  0.2281, -0.2484],
        [-0.3592,  0.1807, -0.2933, -0.2188,  0.2301],
        [ 0.2413,  0.3598, -0.2222, -0.2795, -0.1307],
        [-0.3511, -0.1753,  0.2872, -0.2322,  0.0094],
        [ 0.1164,  0.0620,  0.0414, -0.1472, -0.2698],
        [-0.1343,  0.3105, -0.3529,  0.3063, -0.1436],
        [ 0.2085,  0.0502, -0.2477,  0.2870, -0.0683],
        [-0.2910, -0.2569, -0.1903, -0.0875, -0.3270],
        [ 0.1782,  0.2922, -0.2446, -0.0885, -0.3430],
        [-0.0839,  0.0179,  0.2307, -0.1886,  0.0091],
        [-0.2154, -0.2445,  0.2925,  0.1994, -0.3176],
        [-0.0218, -0.1832, -0.2908,  0.3639, -0.3595],
        [ 0.1016, -0.0547, -0.3834,  0.2332

커뮤니티의 라벨은 `'club'`이라는 노드 피처에 저장되어 있습니다. (0은 instructor의 커뮤니티, 1은 club president의 커뮤니티).  
오로지 0과 33번 노드에만 라벨링 되어 있습니다.

In [6]:
labels = g.ndata['club']
labeled_nodes = [0, 33]
print('Labels', labels[labeled_nodes])

Labels tensor([0, 1], device='cuda:0')


## GraphSAGE 모델 정의하기

우리의 모델은 2개 레이어로 구성되어 있는데, 각각 새로운 노드 표현(representation)을 이웃의 정보를 통합함으로써 계산합니다.  
수식은 다음과 같습니다.  


$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \sigma\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

DGL은 많은 유명한 이웃 통합(neighbor aggregation) 모듈의 구현체를 제공합니다. 모두 쉽게 한 줄의 코드로 호출하여 사용할 수 있습니다.  
지원되는 모델의 전체 리스트는 [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv)에서 보실 수 있습니다.

In [7]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# 2개의 레이어를 가진 GraphSAGE 모델 구축
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, num_classes, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# 주어진 차원의 모델 생성
# 인풋 레이어 차원: 5, 노드 임베딩
# 히든 레이어 차원: 16
# 아웃풋 레이어 차원: 2, 클래스가 2개 있기 때문, 0과 1

net = GraphSAGE(5, 16, 2)

네트워크를 GPU에 복사함

In [8]:
net = net.to('cuda:0')

In [9]:
# ----------- 3. set up loss and optimizer -------------- #
# 이 경우, 학습 루프의 손실

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), node_embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    logits = net(g, inputs)
    
    # 손실 계산
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes])
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.8203792572021484
In epoch 5, loss: 0.4111963212490082
In epoch 10, loss: 0.21308189630508423
In epoch 15, loss: 0.08275498449802399
In epoch 20, loss: 0.03401576727628708
In epoch 25, loss: 0.01440766267478466
In epoch 30, loss: 0.006833690218627453
In epoch 35, loss: 0.0037461717147380114
In epoch 40, loss: 0.0023471189197152853
In epoch 45, loss: 0.0016530591528862715
In epoch 50, loss: 0.0012706018751487136
In epoch 55, loss: 0.0010407611262053251
In epoch 60, loss: 0.0008915828075259924
In epoch 65, loss: 0.0007876282907091081
In epoch 70, loss: 0.000710575666744262
In epoch 75, loss: 0.0006503689801320434
In epoch 80, loss: 0.000602009822614491
In epoch 85, loss: 0.0005602584569714963
In epoch 90, loss: 0.0005215413984842598
In epoch 95, loss: 0.000484131567645818


In [10]:
# ----------- 5. check results ------------------------ #
pred = torch.argmax(logits, axis=1)
print('Accuracy', (pred == labels).sum().item() / len(pred))

Accuracy 0.9411764705882353


**한 GPU 메모리에 그래프와 피처 데이터를 적재할 수 없으면 어떻게 하나요?** 

* GNN을 천제 그래프에 대해 수행하는 대신에, 몇몇 subgraph에 대해 수행해 수렴시켜보세요.
* 다른 샘플을 다른 GPU에 올림으로써 더 빠른 가속을 경험해 보세요.
* 그래프를 여러 머신에 분할하여 분산된 형태로 학습시켜보세요.

추후에 이러한 방법론을 각각 살펴볼 예정입니다.